<a href="https://colab.research.google.com/github/Vhooker/stc510-mod-5-6/blob/main/STC510_Module_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Module 5: Look at Jeopardy words to determine the value of the questions.

Steps- Combine Jeopardy answers and questions to gather all words.
       Clean normalize words and remove punctuation.
       convert values into high and low.


Imports and download tools for data cleaning

In [1]:
import json
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import download


In [2]:
download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Loadning JSON file

In [ ]:
with open('jeopardy.json', 'r') as f:
    jeopardydata = json.load(f)

Make it a dataframe

In [10]:
jeop = pd.DataFrame(jeopardydata)


Remove unnecessary columns. We only have to

In [11]:
jeparody = jeop.drop(['category', 'air_date', 'round', 'show_number'], axis=1)

Combine questions and answer into new column- 'words'

In [12]:
jeparody['words'] = jeparody['question'] + jeparody['answer']

Make the value column a float value-remove $ and convert.

In [13]:
jeparody['value'] = jeparody['value'].str.replace('$', '')
jeparody['value'] = pd.to_numeric(jeparody['value'], errors='coerce')

<ipython-input-13-6a93b8ec3be1>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  jeparody['value'] = jeparody['value'].str.replace('$', '')


Remove rows without values

In [14]:
jeparody = jeparody.dropna(subset=['value'])

Tokenize the questions/answers. Makes the sentences into word lists.  

In [15]:
jeparody['clean words'] = jeparody['words'].apply(lambda x: word_tokenize(x.lower()))

Define stopwords

In [16]:
english_stopwords = set(stopwords.words('english') + ['~', ',', '.', '?', '!', '$', '&', ':', ';', '""', "''", '@'])

Remove stopwords. This cleans the word list to avoid less important but frequent words (e.g. the, and) from skewing the resutls.

In [17]:
jeparody['stopped'] = jeparody['clean words'].apply(lambda x: [word for word in x if word not in english_stopwords])

lemmatization of the word lists to normalize the words (e.g tense)

In [18]:
lemmatizer = WordNetLemmatizer()
jeparody['lemming'] = jeparody['stopped'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

Put the word lists into their own strings for processing

In [19]:
jeparody['finalform'] = jeparody['lemming'].apply(lambda x: ' '.join(x))

Oh yeah! I initially forgot this the accuracy was not great with a full value spread. Convert the values into High (1) and low (0) sections. The values above 700 were high. I haven't watched Jeoparody in a while and with "doubles" and probably bonus rounds I figured I'd catch a majority of the high values and exclued the low values if I made the split 700 even if that left out the 400 and 500 options for the regular game play.

In [20]:
def label_high_value(x):
    if x >= 700:
        return 1
    else:
        return 0
jeparody['highval'] = jeparody['value'].apply(label_high_value)

Create the the dataframe to run the predictions and classification on.

In [21]:
 finaljeopardy= jeparody[['highval', 'finalform']].copy()

In [22]:
print(finaljeopardy)

        highval                                          finalform
0             0  'for last 8 year life galileo house arrest esp...
1             0  'no 2 1912 olympian football star carlisle ind...
2             0  'the city yuma state record average 4,055 hour...
3             0  'in 1963 live `` art linkletter show company s...
4             0  'signer dec. indep. framer constitution mass. ...
...         ...                                                ...
216924        1  'in 2006 cast long-running hit embarked < href...
216925        1  'this puccini opera turn solution 3 riddle pos...
216926        1  'in north america term properly applied 4 spec...
216927        1  'in penny lane `` hellraiser grew barber shave...
216928        1  'from ft. sill okla. made plea arizona land ho...

[204903 rows x 2 columns]


Imports sklearn to do the heavy lifting in analysis of the word patterns.

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

Assign the (x(q/a),y(value#) for the testing and training.

In [24]:
X_train, X_test, y_train, y_test = train_test_split(finaljeopardy['finalform'], finaljeopardy['highval'], random_state=1)

Convert question and answer data into vectors for training and testing datasets

In [25]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_tf = tfidf_vectorizer.fit_transform(X_train)
X_test_tf = tfidf_vectorizer.transform(X_test)

Run the Niave Bayes classification training then run the test data to make predicitions.

In [26]:
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_tf, y_train)
predictions = naive_bayes.predict(X_test_tf)

The accuracy of the predicitons to look at the words in the jeoparody questions/answers and determine the high low values.... better than flipping a coin but not a great parachute.

In [27]:
print ('Accuracy:', accuracy_score(y_test, predictions))

Accuracy: 0.6157224846757506
